In [20]:
import numpy as np
import pandas as pd
from os import listdir
import networkx as nx
from copy import deepcopy
import matplotlib.pyplot as plt
import Compute_Scaffold_Coverages_Utility as cc
from importlib import reload
import sqlite3

def Load_Read_Coverage_and_Assembly_Graph(graphpath, covpath):
    G = nx.read_gml(graphpath)
    nodes = list(G.nodes())
    df_coverage = pd.DataFrame()
    it = 0
    df_coverage_chunks = pd.read_csv(covpath,names = ['Contig','Loc','coverage'], 
                              sep = '\t', low_memory = True, memory_map = True, 
                              dtype = {'Contig': str, 'Loc': 'int32', 'coverage': 'int32'},
                              engine='c', chunksize = 500000, index_col = ['Contig'])
    for chunk in df_coverage_chunks:
        it += 1
        chunk.index = chunk.index.astype('str')
        nodes_pres = list(set(chunk.index.tolist()).intersection(set(nodes)))
        nodes_pres.sort()
        if len(nodes_pres) > 0:
            temp = chunk.loc[nodes_pres]
            temp = temp.reset_index()
            df_coverage = pd.concat([df_coverage, temp])
    df_coverage['Loc'] = df_coverage['Loc']-1
    df_coverage = df_coverage.set_index('Contig')
    print(df_coverage.info())
    return df_coverage, G

In [11]:
graphspath = '/Users/harihara/Mount/MetaCarvel_paper/hmp_scaffolds/stool/'
covpath = '/Users/harihara/Research-Activities/Data/Binnacle-Op/genomecov_d/'

In [12]:
sample = 'SRS104311'

In [13]:
Graph_path= graphspath+sample+'/'+sample+'_scaffolds/oriented.gml'

In [14]:
reload(cc)

<module 'Compute_Scaffold_Coverages_Utility' from '/Users/harihara/Research-Activities/Codes/binnacle/src/Compute_Scaffold_Coverages_Utility.py'>

In [ ]:
%%time
Load_Read_Coverage_and_Assembly_Graph(Graph_path, covpath+sample+'.txt')